In [1]:
!mkdir gcs
!gcsfuse --implicit-dirs --rename-dir-limit=100 --max-conns-per-host=100 "hms_applied_cv" "/home/jupyter/gcs"

mkdir: cannot create directory ‘gcs’: File exists
{"time":"10/04/2024 09:14:14.894125","severity":"INFO","message":"Start gcsfuse/2.0.0 (Go version go1.22.1) for app \"\" using mount point: /home/jupyter/gcs\n"}
{"time":"10/04/2024 09:14:14.894497","severity":"INFO","message":"GCSFuse mount command flags: {\"AppName\":\"\",\"Foreground\":false,\"ConfigFile\":\"\",\"MountOptions\":{},\"DirMode\":493,\"FileMode\":420,\"Uid\":-1,\"Gid\":-1,\"ImplicitDirs\":true,\"OnlyDir\":\"\",\"RenameDirLimit\":100,\"CustomEndpoint\":null,\"BillingProject\":\"\",\"KeyFile\":\"\",\"TokenUrl\":\"\",\"ReuseTokenFromUrl\":true,\"EgressBandwidthLimitBytesPerSecond\":-1,\"OpRateLimitHz\":-1,\"SequentialReadSizeMb\":200,\"MaxRetrySleep\":30000000000,\"StatCacheCapacity\":20460,\"StatCacheTTL\":60000000000,\"TypeCacheTTL\":60000000000,\"HttpClientTimeout\":0,\"MaxRetryDuration\":-1000000000,\"RetryMultiplier\":2,\"LocalFileCache\":false,\"TempDir\":\"\",\"ClientProtocol\":\"http1\",\"MaxConnsPerHost\":100,\"Max

In [2]:
# !pip install xgboost

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import joblib

In [10]:
old_features = pd.read_csv('gcs/hms-harmful-brain-activity-classification/cleaned_train.csv')
new_features = pd.read_csv('gcs/hms-harmful-brain-activity-classification/new_features.csv')

In [11]:
old_features

,Unnamed: 0,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,concatenated_scores,is_center
0,40927,568657,0,0.0,789577333,0,0.0,1825637311,20654,Other,0,0,3,0,2,7,3027,False
1,40928,568657,1,6.0,789577333,1,6.0,3640441665,20654,Other,0,0,3,0,2,7,3027,True
2,40929,568657,2,12.0,789577333,2,12.0,1364530340,20654,Other,0,0,3,0,2,7,3027,False
3,40930,568657,3,16.0,789577333,3,16.0,1874525225,20654,Other,0,0,3,0,2,7,3027,False
4,77116,582999,0,0.0,1552638400,0,0.0,1722186807,20230,LPD,0,12,0,1,0,1,120101,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106795,33556,4294858825,1,2.0,657299228,1,2.0,2581506921,4312,Other,0,0,0,0,1,14,114,False
106796,33557,4294858825,2,6.0,657299228,2,6.0,3251917981,4312,Other,0,0,0,0,1,14,114,True
106797,33558,4294858825,3,8.0,657299228,3,8.0,3646714202,4312,Other,0,0,0,0,1,14,114,False
106798,33559,4294858825,4,12.0,657299228,4,12.0,3219273381,4312,Other,0,0,0,0,1,14,114,False


In [12]:
new_features

,Unnamed: 0,eeg_id,LP__variance_larger_than_standard_deviation,LP__has_duplicate_max,LP__has_duplicate_min,LP__has_duplicate,LP__sum_values,LP__abs_energy,LP__mean_abs_change,LP__mean_change,...,RR__fourier_entropy__bins_5,RR__fourier_entropy__bins_10,RR__fourier_entropy__bins_100,RR__permutation_entropy__dimension_3__tau_1,RR__permutation_entropy__dimension_4__tau_1,RR__permutation_entropy__dimension_5__tau_1,RR__permutation_entropy__dimension_6__tau_1,RR__permutation_entropy__dimension_7__tau_1,RR__query_similarity_count__query_None__threshold_0.0,RR__mean_n_absolute_max__number_of_maxima_7
0,0,3303942477,1.0,0.0,0.0,0.0,-7.351676e+03,2.585435e+07,3.216373,-0.015056,...,0.136002,0.181214,0.316475,1.250239,1.877822,2.569228,3.285148,4.004266,NaN,1837.969849
1,1,1241196880,1.0,0.0,0.0,1.0,-1.618506e+06,2.804918e+08,0.550302,-0.000008,...,0.090729,0.136002,0.181214,0.736680,0.782385,0.829194,0.876541,0.925040,NaN,129.277782
2,2,4065077588,1.0,0.0,0.0,1.0,2.958882e+04,3.198907e+07,5.041877,0.006413,...,0.249958,0.339942,0.771130,1.111769,1.568488,2.065099,2.596294,3.152358,NaN,199.619877
3,3,591800672,1.0,0.0,0.0,1.0,2.480673e+05,1.133460e+07,0.506546,-0.000186,...,0.090729,0.136002,0.316475,1.015282,1.371102,1.752752,2.134890,2.505127,NaN,66.791200
4,4,135276897,1.0,0.0,0.0,1.0,3.874901e+05,3.658458e+07,2.312719,-0.001305,...,0.136002,0.181214,0.226363,1.017677,1.371594,1.747652,2.136593,2.526643,NaN,176.713126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,1122,3561293805,1.0,0.0,0.0,1.0,3.204402e+05,2.558157e+07,1.791134,-0.002983,...,0.170467,0.215617,0.425614,0.926641,1.172225,1.429753,1.702362,1.993302,NaN,94.088824
1123,1123,1648429038,1.0,0.0,0.0,1.0,1.407250e+05,4.060240e+07,1.173372,-0.003972,...,0.181214,0.260704,0.440417,1.042726,1.416579,1.821108,2.252619,2.702940,NaN,210.652326
1124,1124,2541422575,1.0,0.0,0.0,1.0,-1.058758e+04,1.111806e+07,2.055331,-0.002904,...,0.200814,0.245901,0.451164,1.030374,1.395120,1.794200,2.209298,2.628546,NaN,221.973121
1125,1125,974325748,1.0,0.0,0.0,0.0,3.752705e+04,2.989162e+07,2.911714,-0.002687,...,0.090729,0.170467,0.350689,1.083505,1.515875,1.973949,2.424638,2.842376,NaN,220.804980


In [13]:
if 'expert_consensus' in new_features.columns:
    print("expert_consensus present")
else:
    print("expert_consensus NOT present")

expert_consensus NOT present


In [14]:
old_features_unique = old_features.drop_duplicates(subset=['eeg_id'], keep='first')
combined_features = pd.DataFrame()
base_path = 'gcs/hms-harmful-brain-activity-classification/new_features'
for i in range(29):
    new_features_path = f'{base_path}{i}.csv'
    new_features = pd.read_csv(new_features_path)
    temp_combined = pd.merge(new_features, old_features_unique[['eeg_id', 'expert_consensus']], on='eeg_id', how='left')
    
    combined_features = pd.concat([combined_features, temp_combined], ignore_index=True)

# Remove duplicates from the combined dataset
combined_features = combined_features.drop_duplicates(subset=['eeg_id'], keep='first')

In [15]:
combined_features

,Unnamed: 0,eeg_id,LL__variance_larger_than_standard_deviation,LL__has_duplicate_max,LL__has_duplicate_min,LL__has_duplicate,LL__sum_values,LL__abs_energy,LL__mean_abs_change,LL__mean_change,...,RR__fourier_entropy__bins_10,RR__fourier_entropy__bins_100,RR__permutation_entropy__dimension_3__tau_1,RR__permutation_entropy__dimension_4__tau_1,RR__permutation_entropy__dimension_5__tau_1,RR__permutation_entropy__dimension_6__tau_1,RR__permutation_entropy__dimension_7__tau_1,RR__query_similarity_count__query_None__threshold_0.0,RR__mean_n_absolute_max__number_of_maxima_7,expert_consensus
0,0,2022637570,1.0,0.0,0.0,1.0,-235404.787612,2.818751e+07,1.261950,0.006808,...,0.136002,0.290926,1.048271,1.436023,1.853268,2.296737,2.763488,NaN,124.670077,Other
1,1,1376681989,1.0,0.0,0.0,1.0,-128191.644260,1.939504e+08,2.068613,-0.003936,...,0.136002,0.414868,1.163689,1.692133,2.267913,2.866488,3.465324,NaN,549.654811,Seizure
2,2,4183293965,1.0,0.0,0.0,1.0,-266079.324068,3.250886e+07,1.712453,0.000145,...,0.170467,0.271451,0.774179,0.857372,0.943669,1.031855,1.122134,NaN,2389.748431,Other
3,3,3758850061,1.0,0.0,0.0,1.0,247886.286253,2.236191e+07,1.687947,0.005409,...,0.170467,0.395585,0.996659,1.326278,1.678112,2.050456,2.437695,NaN,146.017410,GRDA
4,4,80871439,1.0,0.0,0.0,1.0,309935.319810,1.608429e+07,0.080873,0.000011,...,0.045395,0.136002,0.711625,0.729608,0.747553,0.765722,0.783888,NaN,18.630586,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14737,495,923430193,1.0,0.0,0.0,1.0,-129205.373539,2.492942e+07,2.653488,0.006170,...,0.181214,0.433549,1.114466,1.576127,2.076517,2.612033,3.175263,NaN,173.878595,LRDA
14738,496,4160351544,1.0,0.0,0.0,0.0,-55658.027515,2.808226e+07,0.970477,-0.000176,...,0.136002,0.226363,0.865015,1.052717,1.249729,1.456777,1.672629,NaN,224.836145,Other
14739,497,3532975424,1.0,0.0,0.0,0.0,62400.981025,9.745920e+06,2.033677,0.001133,...,0.136002,0.271451,1.239656,1.852065,2.484379,3.109438,3.645032,NaN,215.161094,LPD
14740,498,1940221249,1.0,0.0,0.0,1.0,-318321.945857,2.852994e+07,3.792420,-0.000124,...,0.136002,0.181214,0.921035,1.158676,1.384665,1.602499,1.801028,NaN,127.673320,GRDA


In [16]:
# # only keep one row for each eeg_id
# old_features_unique = old_features.drop_duplicates(subset=['eeg_id'], keep='first')
# # joining on eeg_id to get the expert_consensus
# combined_features = pd.merge(new_features, old_features_unique[['eeg_id', 'expert_consensus']], on='eeg_id', how='left')
# combined_features

In [17]:
has_nan = combined_features['expert_consensus'].isna().any()

if has_nan:
    print("NaN values in expert_consensus column")
else:
    print("NO NaN values in expert_consensus column")

NO NaN values in expert_consensus column


In [18]:
# the features
x = combined_features.iloc[:, 2:-1]
y = combined_features['expert_consensus']
print(y)
x

0          Other
1        Seizure
2          Other
3           GRDA
4          Other
          ...   
14737       LRDA
14738      Other
14739        LPD
14740       GRDA
14741        LPD
Name: expert_consensus, Length: 14500, dtype: object


,LL__variance_larger_than_standard_deviation,LL__has_duplicate_max,LL__has_duplicate_min,LL__has_duplicate,LL__sum_values,LL__abs_energy,LL__mean_abs_change,LL__mean_change,LL__mean_second_derivative_central,LL__median,...,RR__fourier_entropy__bins_5,RR__fourier_entropy__bins_10,RR__fourier_entropy__bins_100,RR__permutation_entropy__dimension_3__tau_1,RR__permutation_entropy__dimension_4__tau_1,RR__permutation_entropy__dimension_5__tau_1,RR__permutation_entropy__dimension_6__tau_1,RR__permutation_entropy__dimension_7__tau_1,RR__query_similarity_count__query_None__threshold_0.0,RR__mean_n_absolute_max__number_of_maxima_7
0,1.0,0.0,0.0,1.0,-235404.787612,2.818751e+07,1.261950,0.006808,-5.832519e-04,-22.397270,...,0.090729,0.136002,0.290926,1.048271,1.436023,1.853268,2.296737,2.763488,NaN,124.670077
1,1.0,0.0,0.0,1.0,-128191.644260,1.939504e+08,2.068613,-0.003936,5.376103e-04,-7.253205,...,0.090729,0.136002,0.414868,1.163689,1.692133,2.267913,2.866488,3.465324,NaN,549.654811
2,1.0,0.0,0.0,1.0,-266079.324068,3.250886e+07,1.712453,0.000145,6.967698e-06,-20.048945,...,0.090729,0.170467,0.271451,0.774179,0.857372,0.943669,1.031855,1.122134,NaN,2389.748431
3,1.0,0.0,0.0,1.0,247886.286253,2.236191e+07,1.687947,0.005409,-5.511650e-04,20.297157,...,0.090729,0.170467,0.395585,0.996659,1.326278,1.678112,2.050456,2.437695,NaN,146.017410
4,1.0,0.0,0.0,1.0,309935.319810,1.608429e+07,0.080873,0.000011,-5.532418e-09,34.680075,...,0.045395,0.045395,0.136002,0.711625,0.729608,0.747553,0.765722,0.783888,NaN,18.630586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14737,1.0,0.0,0.0,1.0,-129205.373539,2.492942e+07,2.653488,0.006170,2.224127e-06,-14.884228,...,0.136002,0.181214,0.433549,1.114466,1.576127,2.076517,2.612033,3.175263,NaN,173.878595
14738,1.0,0.0,0.0,0.0,-55658.027515,2.808226e+07,0.970477,-0.000176,-6.539890e-06,-6.657038,...,0.125256,0.136002,0.226363,0.865015,1.052717,1.249729,1.456777,1.672629,NaN,224.836145
14739,1.0,0.0,0.0,0.0,62400.981025,9.745920e+06,2.033677,0.001133,-2.080432e-04,8.310953,...,0.090729,0.136002,0.271451,1.239656,1.852065,2.484379,3.109438,3.645032,NaN,215.161094
14740,1.0,0.0,0.0,1.0,-318321.945857,2.852994e+07,3.792420,-0.000124,-1.370323e-06,-34.552313,...,0.090729,0.136002,0.181214,0.921035,1.158676,1.384665,1.602499,1.801028,NaN,127.673320


In [19]:
# 80/20 split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

# convert labels to numbers
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [20]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

model.fit(x_train, y_train_encoded)
predictions_encoded = model.predict(x_test)

accuracy = accuracy_score(y_test_encoded, predictions_encoded)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Get probability distribution for the classes
probabilities = model.predict_proba(x_test)
print("Probability distributions for the first few predictions:")
print(probabilities[:5])

# Save the model and the label encoder
joblib.dump(model, 'xgboost_model.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')
print("Model and Label Encoder saved.")

Accuracy: 65.86%
Probability distributions for the first few predictions:
[[7.0075330e-04 2.1142922e-01 1.2423501e-02 6.4941049e-02 5.0993973e-01
  2.0056573e-01]
 [2.5302151e-04 3.7447686e-04 1.3628592e-02 3.9269566e-03 8.1722826e-01
  1.6458865e-01]
 [4.1299495e-03 6.8679481e-04 8.4141207e-01 5.4110284e-04 1.3538481e-01
  1.7845286e-02]
 [4.0671765e-04 5.5099733e-04 8.9910543e-01 1.1068992e-03 7.0917465e-02
  2.7912464e-02]
 [2.7192805e-03 5.7077086e-01 3.1766992e-02 9.0948343e-03 2.7850232e-01
  1.0714570e-01]]
Model and Label Encoder saved.


In [21]:
# Load the Label Encoder
label_encoder = joblib.load('label_encoder.pkl')

label_mapping = {}
for index, label in enumerate(label_encoder.classes_):
    label_mapping[label] = index
    
print("Label Encoder mapping:")
for label in label_mapping:
    print(f"'{label}' is encoded as {label_mapping[label]}")

Label Encoder mapping:
'GPD' is encoded as 0
'GRDA' is encoded as 1
'LPD' is encoded as 2
'LRDA' is encoded as 3
'Other' is encoded as 4
'Seizure' is encoded as 5


In [22]:
def kl_divergence(p, q):
    # Avoid division by zero
    p = np.where(p == 0, 1e-10, p)
    q = np.where(q == 0, 1e-10, q)
    return np.sum(p * np.log(p / q), axis=1)

num_classes = probabilities.shape[1]
y_test_one_hot = np.eye(num_classes)[y_test_encoded]

kl_divergences = kl_divergence(y_test_one_hot, probabilities)

average_kl_divergence = np.mean(kl_divergences)
print(f"Average KL Divergence: {average_kl_divergence}")

Average KL Divergence: 1.0610615666311134
